## Get Tweets in Real Time using Twitter API

In [17]:
import tweepy

from tweepy import OAuthHandler

from tweepy import Stream

from tweepy.streaming import StreamListener

In [18]:
import socket
import json

In [19]:
consumer_key = 'HMNx5t9S6PJqglnb9kboYtZgq'
consumer_secret = 'kQMnxYlO1BFGjccvWiTLt69QdrYIEWE1X4iKz0MQqRnbrBHEPQ'
access_token = '3112767474-K2tNQiPx8MekqmqFZ4thh0OS5q4zkrAoJDY8NqC'
access_secret = '93oXUB570inlg8nO4foBTxgcuo2QLuGfJnbIIKwSfCdBl'

In [20]:
class TweetsListener (StreamListener):
    def __init__ (self,csocket):
        self.client_socket = csocket
        
        def on_data(self, data):
            try:
                
                msg = json.loads(data)
                print(msg['text'].encoder('utf-8'))
                
                self.client_socket.send(msg['text'].encode('utf-8'))
                return True
        
            except BaseException as e:
                print("Error on_data: %s" % str(e))
                return True
        
        def on_error(self, status):
            print(status)
            return True
                

In [21]:
def sendData(c_socket):
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    twitter_stream = Stream(auth, TwitterListener(c_socket))
    twitter_stream.filter(trace=['machinelearning'])

In [22]:
s = socket.socket()
host = "127.0.0.1"
port = 9999
s.bind((host,port))
print("Listening on port: %s" %str(port))

Listening on port: 9999


In [ ]:
s.listen(5)
c, addr = s.accept()
print("Received request from:" + str(addr))

In [ ]:
sendData(c)